In [1]:
import findspark
findspark.init('/Users/dreyco676/spark-1.6.0-bin-hadoop2.6/')

import pyspark
from pyspark.sql import SQLContext
import langid
from nltk.stem.wordnet import WordNetLemmatizer
import string
from pyspark.sql.types import *
import re
from nltk.corpus import stopwords
from pyspark.sql import SQLContext
from pyspark.sql.functions import udf
from nltk import pos_tag, word_tokenize

sc = pyspark.SparkContext(appName="tweet_classifier")
sqlContext = SQLContext(sc)

In [2]:
# Load a text file and convert each line to a Row.
lines = sc.textFile("/Users/dreyco676/nlp_spark/data/classified_tweets.txt")
parts = lines.map(lambda l: l.split("\t"))
# Filter bad rows out
garantee_col = parts.filter(lambda l: len(l) == 2)
training = garantee_col.map(lambda p: (p[0], p[1].strip()))
# Create DataFrame
training_df = sqlContext.createDataFrame(training, ["tweet", "classification"])

In [7]:
# Use langid module to classify the language to make sure we are applying the correct cleanup actions
# https://github.com/saffsd/langid.py
def check_lang(data_str):
    predict_lang = langid.classify(data_str)
    if predict_lang[1] >= .9:
        language = predict_lang[0]
    else:
        language = 'NA'
    return language

check_lang_udf = udf(check_lang, StringType())

In [ ]:
# Stop words usually refer to the most common words in a language, there is no single universal list of stop words used
# by all natural language processing tools.
# Reduces Dimensionality
# removes stop words of a single Tweets (cleaned_str/row/document)
def remove_stops(data_str):
    # expects a string
    stops = set(stopwords.words("english"))
    list_pos = 0
    cleaned_str = ''
    text = data_str.split()
    for word in text:
        if word not in stops:
            # rebuild cleaned_str
            if list_pos == 0:
                cleaned_str = word
            else:
                cleaned_str = cleaned_str + ' ' + word
            list_pos += 1
    return cleaned_str

remove_stops_udf = udf(remove_stops, StringType())

In [ ]:
# catch-all to remove other 'words' that I felt didn't add a lot of value
# Reduces Dimensionality, gets rid of a lot of unique urls
def remove_features(data_str):
    # compile regex
    url_re = re.compile('https?://(www.)?\w+\.\w+(/\w+)*/?')
    punc_re = re.compile('[%s]' % re.escape(string.punctuation))
    num_re = re.compile('(\\d+)')
    mention_re = re.compile('@(\w+)')
    alpha_num_re = re.compile("^[a-z0-9_.]+$")
    # convert to lowercase
    data_str = data_str.lower()
    # remove hyperlinks
    data_str = url_re.sub(' ', data_str)
    # remove @mentions
    data_str = mention_re.sub(' ', data_str)
    # remove puncuation
    data_str = punc_re.sub(' ', data_str)
    # remove numeric 'words'
    data_str = num_re.sub(' ', data_str)
    # remove non a-z 0-9 characters and words shorter than 3 characters
    list_pos = 0
    cleaned_str = ''
    for word in data_str.split():
        if list_pos == 0:
            if alpha_num_re.match(word) and len(word) > 2:
                cleaned_str = word
            else:
                cleaned_str = ' '
        else:
            if alpha_num_re.match(word) and len(word) > 2:
                cleaned_str = cleaned_str + ' ' + word
            else:
                cleaned_str = cleaned_str + ' '
        list_pos += 1
    return cleaned_str

remove_features_udf = udf(remove_features, StringType())

In [13]:
def tag_and_remove(data_str):
    cleaned_str = ' '
    # noun tags
    nn_tags = ['NN', 'NNP', 'NNP', 'NNPS', 'NNS']
    # adjectives
    jj_tags = ['JJ', 'JJR', 'JJS']
    # verbs
    vb_tags = ['VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ']
    nltk_tags = nn_tags + jj_tags + vb_tags
    
    # break string into 'words'
    text = data_str.split()
    
    # tag the text and keep only those with the right tags
    tagged_text = pos_tag(text)
    for tagged_word in tagged_text:
        if tagged_word[1] in nltk_tags:
            cleaned_str += tagged_word[0] + ' '
            
    return cleaned_str

tag_and_remove_udf = udf(tag_and_remove, StringType())

In [ ]:
# Tweets are going to use different forms of a word, such as organize, organizes, and
# organizing. Additionally, there are families of derivationally related words with similar meanings, such as democracy,
# democratic, and democratization. In many situations, it seems as if it would be useful for a search for one of these
# words to return documents that contain another word in the set.
# Reduces Dimensionality and boosts numerical measures like TFIDF

# http://nlp.stanford.edu/IR-book/html/htmledition/stemming-and-lemmatization-1.html
# lemmatization of a single Tweets (cleaned_str/row/document)
def lemmatize(data_str):
    # expects a string
    list_pos = 0
    cleaned_str = ''
    lmtzr = WordNetLemmatizer()
    text = data_str.split()
    for word in text:
        lemma = lmtzr.lemmatize(word)
        if list_pos == 0:
            cleaned_str = lemma
        else:
            cleaned_str = cleaned_str + ' ' + lemma
        list_pos += 1
    return cleaned_str

lemmatize_udf = udf(lemm_verbs, StringType())

In [12]:
# predict language and filter out those with less than 90% chance of being English
lang_df = training_df.withColumn("lang", check_lang_udf("tweet"))
en_df = lang_df.filter(lang_df["lang"] == 'en')

In [ ]:
# remove stop words to reduce dimensionality
rm_stops_df = en_df.withColumn("tweet", remove_stops_udf("tweet"))

In [ ]:
# remove other non essential words, think of it as my personal stop word list
rm_features_df = rm_stops_df.withColumn("tweet", remove_features_udf("tweet"))

In [ ]:
# tag the words remaining and keep only Nouns, Verbs and Adjectives
tagged_df = rm_features_df.withColumn("tweet", tag_and_remove_udf("tweet"))

In [ ]:
# lemmatization of remaining words to reduce dimensionality & boost measures
lemm_df = tagged_df.withColumn("tweet", lemmatize_udf("tweet"))